In [84]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from datetime import datetime
import time
import copy

# data preprocessing

In [2]:
dataset = pd.read_csv("iris/iris.csv", header=None)

In [4]:
def encode_textdata(ds, column):
    ids = {name: id for name, id in zip(list(ds[column].unique()), range(ds.shape[0]))}
    return ids, ds[column].transform(lambda x: ids[x])

In [5]:
ids, dataslice = encode_textdata(dataset, 4)

In [6]:
x = dataset[[0, 1, 2, 3]]

In [16]:
scaler = Normalizer()
scaler.fit(x)
X = scaler.fit_transform(x)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, dataslice, test_size=0.3, random_state=42)

In [75]:
y_train = y_train.to_numpy()
y_train = y_train.reshape([y_train.shape[0], 1])
y_test = y_test.to_numpy()
y_test = y_test.reshape([y_test.shape[0], 1])

# Model

In [66]:
class Layer():
    def __init__(self, input_shape, out_shape):
        self.W = np.random.randn(input_shape, out_shape)
        self.b = np.random.randn(1, out_shape)

    def forward(self, x):
        return np.matmul(x**2, self.W) + self.b

In [67]:
class GAModel():
    def __init__(self, layers_shapes):
        self.layers = list()
        for i in range(1, len(layers_shapes)):
            self.layers.append(Layer(layers_shapes[i-1], layers_shapes[i]))

    def mutate(self, lr, scalar):
        for i in self.layers:
            i.W = i.W + lr * scalar * np.random.choice([-1, 1], size=i.W.shape)
            i.b = i.b + lr * scalar * np.random.choice([-1, 1], size=i.b.shape)
        return self

    def softmax(self, x):
        exp_vals = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_vals / np.sum(exp_vals, axis=-1, keepdims=True)
    
    def forward(self, x):
        for i in self.layers:
            x = i.forward(x)
        return self.softmax(x)

# train

In [24]:
def accuracy(y_pred, y):
    y_pred = np.argmax(y_pred, axis=-1, keepdims=True)
    accuracy = np.sum(y_pred == y)/y.shape[0]
    return accuracy

In [26]:
def train_step(model, x, y):
    y_pred = model.forward(x)
    return accuracy(y_pred, y)

In [86]:
def train(models, epochs, x, y, x_test, y_test, lr, scalar, n_children, model_name):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/' + model_name + '_{}'.format(timestamp))
    for epoch in range(0, epochs):
        list_acc = []
        for model in models:
            list_acc.append(train_step(model, x, y))
        best_acc = max(list_acc)
        best_model = models[list_acc.index(max(list_acc))]
        test_acc = accuracy(best_model.forward(x_test), y_test)
        models = [copy.deepcopy(best_model).mutate(lr, scalar)  for _ in range(n_children)]
        writer.add_scalars('Training vs. Validation Acc',
                    { 'Training' : best_acc, 'Validation' : test_acc },
                    epoch + 1)
        print(f"epoch => {epoch}, acc = {best_acc}, test_acc = {test_acc}")
    return best_model

In [90]:
model = GAModel([4, 10, 10, 3])

In [91]:
model = train([model], 200, X_train, y_train, X_test, y_test, 2, 0.1, 1000, "dodel")

epoch => 0, acc = 0.3523809523809524, test_acc = 0.28888888888888886
epoch => 1, acc = 0.638095238095238, test_acc = 0.7111111111111111
epoch => 2, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 3, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 4, acc = 0.6476190476190476, test_acc = 0.6888888888888889
epoch => 5, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 6, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 7, acc = 0.6476190476190476, test_acc = 0.6888888888888889
epoch => 8, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 9, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 10, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 11, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 12, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 13, acc = 0.6476190476190476, test_acc = 0.7111111111111111
epoch => 14, acc = 0.6476190476190476, test_

In [92]:
%load_ext tensorboard

In [ ]:
%reload_ext tensorboard

In [93]:
%tensorboard --logdir runs/ --port=6009